In [4]:
import string
import nltk
import re
from nltk.corpus import stopwords
import pkg_resources
import pickle
import json
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize

C:\Users\csvel\AppData\Local\Temp\ipykernel_20008\3299783850.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [5]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

from random import randint
from pickle import load
from keras.models import load_model
from keras.utils import pad_sequences

In [6]:
#loading file 

def load_doc(filename):
    file= open(filename,'r')

    text= file.read()
    file.close()
    return text

def clean_doc(doc):
    doc=doc.replace('--',' ')
    tokens=doc.split()
    table=str.maketrans('','', string.punctuation)
    tokens= [w.translate(table) for w in tokens]

    tokens =[word for word in tokens if word.isalpha()]

    tokens =[word.lower() for word in tokens]

    return tokens

def save_doc(lines,filename):
    data='\n'.join(lines)
    file=open(filename, 'w')
    file.write(data)
    file.close()

In [9]:
def tokenize(sentences):
    print("Starting Cleaning Process")
    
    tokenized_sentences =[]

    for sentence in tqdm(sentences):

        sentence= cleanhtml(sentence)
        sentence= replace_urls(sentence)
        sentence= remove_email(sentence)
        sentence= re.sub(r'[^a-zA-Z]',' ',sentence)
        sentence= sentence.lower()
        sentence= misc(sentence)

        tokenized_sentences.append(sentence)

    return tokenized_sentences

def cleanhtml(raw_html):
    cleanr=re.compile('<.*?>')
    cleantext= re.sub(cleanr, '', raw_html)
    return cleantext

def replace_urls(data):
    url_pattern=re.compile(r'https?://\S+|www\.\S+')
    data= url_pattern.sub(r'', data)
    return data

def remove_email(data):
    data=re.sub('\S*@\S*\s?', '', data)
    return data

def misc(data):
    data=re.sub('\s+',' ', data)

    data=re.sub("\'","", data)
    data=re.sub("ww+","",data)

    roman= re.compile(r'(\b[MDCLXVI]+\b)(\.)?', re.I)
    data=re.sub(roman,"", data)
    return data  


In [10]:
def littleCleaning(sentences):
    print("Starting cleaning Process")
    ret_list=[]
    for sentence in sentences:
        words= sentence.split(" ")
        if len(words)>5:
            ret_list.append(sentence)
        else:
            continue
    return ret_list

In [11]:
path=('republic.txt')
text= open(path).read().lower()

print('length of the corpus is: ', len(text))

length of the corpus is:  1213703


In [12]:
data_list=text.split(".")
norm_sentence=[]

def norm_pipeline(sentences):
    print("Starting Normalization Process")
    sentences= tokenize(sentences)
    sentences=littleCleaning(sentences)
    print("Normalization process finished")
    return sentences

pro_sentences= norm_pipeline(data_list)

Starting Normalization Process
Starting Cleaning Process


  0%|          | 0/7313 [00:00<?, ?it/s]

Starting cleaning Process
Normalization process finished


In [13]:
len(pro_sentences)

6498

In [14]:
dataText="".join(pro_sentences[:700])


In [16]:
tokens=clean_doc(dataText)
print('Total tokens:', len(tokens))
print('Unique Tokens: ', len(set(tokens)))

Total tokens: 21747
Unique Tokens:  3545


In [18]:
length=50+1

Sequences= list()
for i in range(length,len(tokens)):
    seq= tokens[i-length:i]

    line=' '. join(seq)

    Sequences.append(line)

print('Total Sequences: ', len(Sequences))

Total Sequences:  21696


In [19]:
save_doc(Sequences,filename='republic_sequences.txt')

In [21]:
doc=load_doc('republic_sequences.txt')
lines=doc.split('\n')

In [23]:
tokenizer= Tokenizer()
tokenizer.fit_on_texts(lines)
sequences= tokenizer.texts_to_sequences(lines)

vocab_size=len(tokenizer.word_index)+1

In [24]:
sequences=array(sequences)

X,y =sequences[:,:-1], sequences[:,-1]

y=to_categorical(y,num_classes=vocab_size)
seq_length=X.shape[1]

In [25]:
model=Sequential()
model.add(Embedding(vocab_size,50, input_length=seq_length))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size=128
epochs=50

model.fit(X,y, batch_size=batch_size,epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            177300    
                                                                 
 lstm (LSTM)                 (None, 50, 50)            20200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 3546)              180846    
                                                                 
Total params: 401,096
Trainable params: 401,096
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
170/170 [=========================

In [ ]:
model.save('nexword.h5')
dump(tokenizer, open('tokenizer.pkl','wb'))

In [ ]:
import numpy as np

def generate_seq(model,tokenizer,seq_length, seed,length,n_words):
    result=list()
    in_text=seed_text

    for _ in range(n_words):
        encoded=tokenizer.texts_to_sequences([in_text])[0]
        encoded= pad_sequences([encoded],maxlen=seq_length, truncating='pre')
        predict_x=model.predict(encoded)
        yhat=np.argmax(predict_x,axis=1)

        out_word=''

        for word,index in tokenizer.word_index.items():
            if index == yhat:
                out_word =word
                break
        in_text += ' '+ out_word
        result.append(out_word)
    return ' '.join(result)